<a href="https://colab.research.google.com/github/applejxd/colaboratory/blob/master/Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

数値積分法の精度を円周率の計算
\begin{equation}
    I\equiv\int_a^b dx f(x)
    =\int_{0}^1dx\frac{4}{1+x^2}=\pi
\end{equation}
で比較する。

台形公式は積分を単純な差分で表したもので
\begin{equation}
I\simeq\frac{\Delta x}{2}\left(f(x_0)+2\sum_{k=1}^{n-1}f(x_k)+f(x_n)\right),\quad
\Delta x\equiv\frac{x_n - x_0}{n},\quad x_k\equiv x_0 + k\Delta x
\end{equation}
である。

In [3]:
import numpy as np

#: 被積分関数
def integrand(x: float) -> float:
    return 4 / (1 + x**2)

In [19]:
from typing import Callable

def trapezoid(integrand: Callable[[float], float], 
              x_0: float, x_n: float, num: int) -> float:
    # 分割幅
    dx: float = 1./num
    # 台形公式
    sum: float = dx / 2 * (integrand(x_0) + integrand(x_n))
    for k in range(1, num):
        x_k = x_0 + k * dx
        sum += dx * integrand(x_k)
    return sum

print(trapezoid(integrand, 0, 1, 100))

3.141575986923127


シンプソン公式は 2 次のニュートン・コーツ公式（2 次のラグランジュ補間による積分）で
\begin{equation}
I\simeq\frac{\Delta x}{3}\left(f(x_0)+4\sum_{k=1,3,\cdots}^{2n-1}f(x_k)+2\sum_{k=2,4,\cdots}^{2n-2}f(x_k)+f(x_{2n})\right),\quad
\Delta x\equiv\frac{x_{2n}-x_0}{2n},\quad
x_k\equiv x_0 + k\Delta x
\end{equation}
となる。

In [20]:
def simpson(integrand: Callable[[float], float], 
            x_0: float, x_2n: float, num: int) -> float:
    dx = (x_2n - x_0) / (2 * num)
    sum = dx / 3 * (integrand(x_0) + integrand(x_2n))
    
    #: 奇数の場合
    for k in np.arange(1, 2*num, 2):
        x_k = x_0 + k * dx
        sum += dx * 4/3 * integrand(x_k)
    #: 偶数の場合
    for k in np.arange(2, 2*num, 2):
        x_k = x_0 + k * dx
        sum += dx * 2/3 * integrand(x_k)
    return sum

print(simpson(integrand, 0, 1, 50))

3.141592653589754


ロンバーグ法は収束値が真の積分値であるような収束列を用いて、補外を行うことで積分値を求めるアルゴリズムである。
このような収束列を台形公式によって
\begin{equation}
\begin{split}
&T_0^n\equiv\frac{\Delta x}{2}\left(f(x_0)+2\sum_{k=1}^{2^n-1}f(x_k)+f(x_{2^n})\right)\overset{n\rightarrow\infty}{\rightarrow}I,\\
&\Delta x\equiv\frac{x_{2^n}-x_0}{2^n},\quad x_k\equiv k\Delta x
\end{split}
\end{equation}
のように作る。
上の添字は分割数の添字であり、下の添字は補外回数の添字である。
ロンバーグ法は各 $T^n_0$ に対して
\begin{equation}
T^{n+1}_m \equiv\frac{4^m T^{n+1}_{m-1} - T^n_{m-1}}{4^m-1}
\end{equation}
を計算する。
計算は更新幅が一定以下になるまで行う。

In [22]:
def romberg(integrand: Callable[[float], float], num: int) -> float:
    epsilon: float = 1e-10
    #: インデックスは下添字
    T_list: list = [trapezoid(integrand, 0, 1, 100)]
    for up_index in range(1, num+1):
        #: T_0 のリスト更新
        T_0: float = trapezoid(integrand, 0, 1, up_index)
        # 終了判定
        if abs(T_0 - T_list[-1]) < epsilon:
            return T_0

        #: 2^up_index 分割に対する Romberg 補外
        T_tmp_list: list = [T_0]
        for low_index in range(1, up_index+1):
            T_tmp_list.append(
                (4. ** low_index * T_tmp_list[low_index] - T_list[low_index])
                / (4. ** low_index - 1.)
                )
            #: 終了判定
            if abs(T_tmp_list[-1] - T_tmp_list[-2]) < epsilon:
                return T_tmp_list[-1]
        #: リスト更新
        T_list = T_tmp_list
    return T_list[-1]
            
print(romberg(integrand, 6))

IndexError: ignored

ガウス・ルジャンドル積分